In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import random

from lib.utils import prepare_dataset, tensorsFromPair 
from lib.models import EncoderRNN, AttnDecoderRNN
from lib.train import train
from lib.BB import BB

In [6]:
# !wget https://download.pytorch.org/tutorial/data.zip

In [7]:
# !unzip data.zip

In [8]:
eng, fra, pairs = prepare_dataset('data/eng-fra.txt')
print(random.choice(pairs))

Reading lines...
['tu as vraiment beaucoup de chance .', 'you re very fortunate .']


In [9]:
device = 'cpu'

In [10]:
hidden_size = 256
encoder = EncoderRNN(fra.n_words, hidden_size, device).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, eng.n_words, device, dropout_p=0.1).to(device)

In [7]:
# encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=1e-3)
# decoder_optimizer = torch.optim.SGD(attn_decoder.parameters(), lr=1e-3)

In [11]:
encoder_optimizer = BB(encoder.parameters())
decoder_optimizer = BB(attn_decoder.parameters())

In [ ]:
optimizers = ['BB', 'Adam', 'SGD', 'LBFGS', '']

In [12]:
train_loss = train(encoder, attn_decoder, 75000, encoder_optimizer, 
      decoder_optimizer, pairs, device, source = fra, 
      target = eng, print_every=10)

/home/anvlfilippova/OptML-project/lib/BB.py:113: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  p.data.add_(-group['lr'], d_p)


10 6.272100426355998
20 5.066335162662325


KeyboardInterrupt: 

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]